In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import datetime
import uule_grabber
import time
import secrets
import csv
from random import randint


chromedriver_autoinstaller.install()

while True:
    typedispley = input('d or m - ')
    if typedispley == 'm' or typedispley == 'd':
        break

myshop = 'KENZI'
filegeo = 'geo_u.txt'
with open(f'{filegeo}', 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
        geo = list(map(lambda x: x.replace('\n', ''), proxies1))
lich_geo = len(geo)      
filegeo = 'keywords_kenzi.txt'
with open(f'{filegeo}', 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
        keywords = list(map(lambda x: x.replace('\n', ''), proxies1))

hl = 'ru' #hl=sv          Swedish https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
country = 'ua' #se 	Sweden  
num = '10' # Кількість результатів пошуку
zone = 'com.ua' #'com.ua'
# region = 'Kyiv,Kyiv city,Ukraine'  #'Kyiv,Kyiv city,Ukraine', 'UA'   New York,New York,United States
# uule = uule_grabber.uule(region)
opys = {}
brands = {}
lich = 0
lich4 = 0
for region in geo:
    posmyshop = ''
    uule = uule_grabber.uule(region)
    city = region.split(',')[0]
    keyword = secrets.choice(keywords)

    chrome_options = webdriver.ChromeOptions()
#     chrome_options.headless = True
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    if (typedispley == 'm'):
        mobile_emulation = { "deviceName": "Nexus 5" }
        chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
        lichdisp = 2
    elif (typedispley == 'd'):
        chrome_options.add_argument("start-maximized")
        lichdisp = 4

    dt = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

#     print(city, keyword)
    browser = webdriver.Chrome(options=chrome_options)
    if ('і' in keyword or 'чохол' in keyword):
        hl = 'ua'
    else:
        hl = 'ru'
    browser.get(f'https://google.com.ua/search?q={keyword}&pws=0&uule={uule}&hl={hl}&gl={country}')
    time.sleep(10)
    
#     opys = {}
#     brands = {}
    position = '- Відсутній показ'
    try:
        if (browser.find_elements(By.XPATH, "//div[@id='result-stats']") or browser.find_elements(By.XPATH, "//li[@class='LLC6Eb']")):
            browser.save_screenshot(f'screenshots/{dt}_{city}_desk.png')
            if (typedispley == 'm'):
                brend = browser.find_elements(By.XPATH, "//span[@class='vBKJcb']")
            elif (typedispley == 'd'):
                brend = browser.find_elements(By.XPATH, "//div[@class='LbUacb']")
            brends = list(map(lambda x: x.text, brend))
            if (myshop in brends):
                position = '- Присутній в шопінгу'
                lich = lich + 1
            if (myshop in brends[:lichdisp]):
                position = '- В першому екрані'
                lich4 = lich4 + 1
            if (myshop in brends): 
                posmyshop = f'- позиція {brends.index(myshop) + 1}'
            print(f'{city} - {keyword} {position} {posmyshop}')
#             print(f'{city} - {keyword} {position} - позиція {if (len(brends)>0): brends.index(myshop) + 1}')
            brands[keyword+'_'+city] = brends
#             print(brends)
            if (typedispley == 'm'):
                title = browser.find_elements(By.XPATH, "//div[@class='bXPcId pymv4e']")
            elif (typedispley == 'd'):
                title = browser.find_elements(By.XPATH, "//span[@class='pymv4e']")
            titlesf = list(filter(lambda x: len(x.text) > 0, title))
            titles = list(map(lambda x: x.text, titlesf))
#             print(titles)
#             opys[keyword] = [brends, titles]
            opys[keyword+'_'+city] = [list(tup) for tup in zip(brends, titles)]            
    except Exception as e:
        print(f"The error '{e}' occurred")
        pass


    browser.close()
    browser.quit()
    delay = randint(5,15)
    time.sleep(delay) 
#     break
    
def save_csv(typelist, dict):    
    try:
        file = open(f"{dt_csv}_{typelist}.csv", "w", encoding="utf-8")
        # make write object
        writer = csv.writer(file, lineterminator = '\n')
        for key, value in dict.items():
            writer.writerow([key, value])
        file.close()
        print(f"{dt_csv}_{typelist}.csv - saved")
    except Exception as e:
        print(f"The error '{e}' occurred")
        pass
    
# dt_csv = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")  
# save_csv('vydacha', opys)
# save_csv('brands', brands)
# print ('Присутній в шопінгу: ',(lich*100//lich_geo),"%")
# print ('В першому екрані: ',(lich4*100//lich_geo),"%")

d or m - d
Vinnytsia - redmi note 7 чехол книжка - Відсутній показ 
Lutsk - купить чехол на редми 7 книжка - Присутній в шопінгу - позиція 13
Dnipro - чехол книжка на meizu m6 note - Відсутній показ 
Kryvyi Rih - редми нот 7 чехол книжка - Присутній в шопінгу - позиція 27
Zhytomyr - самсунг а 30 чехол книжка - В першому екрані - позиція 3
Uzhhorod - чехол книжка iphone 11 pro max - В першому екрані - позиція 3
Zaporizhzhia - redmi 9c чехол книжка - Відсутній показ 
Ivano-Frankivsk - samsung galaxy a02 чехол книжка - Присутній в шопінгу - позиція 6
Bila Tserkva - чехол книжка на iphone 6s - В першому екрані - позиція 1
Kyiv - купить чехол книжку на редми 6 - Присутній в шопінгу - позиція 24
Kropyvnytskyi - чехол книжка на редми 8 - Відсутній показ 
Rivne - чехол книжка на редми 8 - Відсутній показ 
Lviv - чехол книжка на хуавей п смарт плюс - В першому екрані - позиція 1
Mykolaiv - купить чехол на самсунг а10 книжка - В першому екрані - позиція 2
Odessa - редми 7а чехол книжка - Відсутн